# Hybrid Demucs from Colab

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)
This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [19]:
# LP: fix for https://github.com/pytorch/pytorch/issues/67089 
%pip install torch==1.10.0+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.7 MB/s eta 0:09:43tcmalloc: large alloc 1147494400 bytes == 0x55b77b1ee000 @  0x7f2c4e608615 0x55b7426ba4cc 0x55b74279a47a 0x55b7426bd2ed 0x55b7427aee1d 0x55b742730e99 0x55b74272b9ee 0x55b7426bebda 0x55b742730d00 0x55b74272b9ee 0x55b7426bebda 0x55b74272d737 0x55b7427afc66 0x55b74272cdaf 0x55b7427afc66 0x55b74272cdaf 0x55b7427afc66 0x55b74272cdaf 0x55b7426bf039 0x55b742702409 0x55b7426bdc52 0x55b742730c25 0x55b74272b9ee 0x55b7426bebda 0x55b74272d737 0x55b74272b9ee 0x55b7426bebda 0x55b74272c915 0x55b7426beafa 0x55b74272cc0d 0x55b74272b9ee
     |██████████████████▌             | 1055.7 MB 1.8 MB/s eta 0:07:08tcmalloc: large alloc 1434370048 bytes == 0x55b7bf844000 @  0x7f2c4e608615 0x55b7426ba4cc 0x55b74279a47a 0x55b7426bd2ed 0x55b7427aee1d 0x55b742730e99 0x55b74272b9ee 0x55b7426bebda 0x55b742730d00 0x55b74272b9ee 0x55b7426bebda 0x55b74272d737 0x55b7427afc66 0x55b7

In [1]:
!pip install tqdm lameenc pyyaml
!pip install julius diffq dora-search
!pip install openunmix

In [7]:
# LP: useful for timing purposes
!pip install ipython-autotime
%load_ext autotime

time: 129 µs (started: 2021-11-15 12:48:39 +00:00)


In [2]:
! rm -rf demucs
! git clone https://github.com/facebookresearch/demucs.git demucs
%cd demucs
! python setup.py install

Cloning into 'demucs'...
remote: Enumerating objects: 4946, done.
remote: Counting objects: 100% (2089/2089), done.
remote: Compressing objects: 100% (1788/1788), done.
remote: Total 4946 (delta 315), reused 1961 (delta 242), pack-reused 2857
Receiving objects: 100% (4946/4946), 76.14 MiB | 22.49 MiB/s, done.
Resolving deltas: 100% (529/529), done.
/content/demucs
running install
running bdist_egg
running egg_info
creating demucs.egg-info
writing demucs.egg-info/PKG-INFO
writing dependency_links to demucs.egg-info/dependency_links.txt
writing entry points to demucs.egg-info/entry_points.txt
writing requirements to demucs.egg-info/requires.txt
writing top-level names to demucs.egg-info/top_level.txt
writing manifest file 'demucs.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'demucs.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating b

In [ ]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!
from google.colab import drive
drive.mount('/gdrive')

In [4]:
# Customize the following options!
mp3 = True
mp3_rate = 320
model = "mdx_extra"
extensions = ["mp3", "wav", "ogg", "flac"]
in_path = '/gdrive/MyDrive/demucs/in'
out_path = '/gdrive/MyDrive/demucs/out/'

In [3]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()
    
    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()
    
    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [5]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

Going to separate the files:
/gdrive/MyDrive/demucs/in/82929740.mp3
With command:  python3 -m demucs.separate -o /gdrive/MyDrive/demucs/out/ -n mdx_extra --mp3 --mp3-bitrate=320


Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/e51eebcc-c1b80bdd.th" to /root/.cache/torch/hub/checkpoints/e51eebcc-c1b80bdd.th
100%|██████████| 160M/160M [00:16<00:00, 10.1MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/a1d90b5c-ae9d2452.th" to /root/.cache/torch/hub/checkpoints/a1d90b5c-ae9d2452.th
100%|██████████| 160M/160M [00:16<00:00, 10.0MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/5d2d6c55-db83574e.th" to /root/.cache/torch/hub/checkpoints/5d2d6c55-db83574e.th
100%|██████████| 160M/160M [00:15<00:00, 10.6MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/cfa93e08-61801ae1.th" to /root/.cache/torch/hub/checkpoints/cfa93e08-61801ae1.th
100%|██████████| 160M/160M [00:17<00:00, 9.72MB/s]

Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /gdrive/MyDrive/demucs/out/mdx_extra
Separating track /gdrive/MyDrive/demucs/in/82929740.mp3



100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.42seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.21seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.26seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.98seconds/s]


In [8]:
# LP: This second command will not download models again!
separate()

Going to separate the files:
/gdrive/MyDrive/demucs/in/82929740.mp3
With command:  python3 -m demucs.separate -o /gdrive/MyDrive/demucs/out/ -n mdx_extra --mp3 --mp3-bitrate=320
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /gdrive/MyDrive/demucs/out/mdx_extra
Separating track /gdrive/MyDrive/demucs/in/82929740.mp3


100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.87seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.26seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 18.34seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 198.0/198.0 [00:10<00:00, 19.12seconds/s]


time: 2min 38s (started: 2021-11-15 12:49:07 +00:00)


In [ ]:
# This is manual upload and download :)
from_upload()
!zip -r separated.zip separated
files.download('./separated.zip')